In this Project we are using Random Forest to Train a model which Generates Xpath for the Html Elements.

In [ ]:
import pandas as pd

df1 = pd.read_csv('CombinedDataset.csv')



print(df1.tail())

                                                column1  \
1738                            article class container   
1739       header id access name webapp class container   
1740  header id access class container name webapp d...   
1741                                    div name webapp   
1742                                   footer id access   

                                                column2  
1738                      //article[@class='container']  
1739  //header[@id='access' and @name='webapp' and @...  
1740  //header[@id='access' and @class='container' a...  
1741                              //div[@name='webapp']  
1742                             //footer[@id='access']  


In [ ]:
import re

def modify_text(text):
  modify = text.replace('/','/ ').replace('[',' [').replace('@',' @ ').replace('=',' = ').replace('\'', ' \' ').replace(']',' ]')
  remodify = re.sub(r'\s+', ' ', modify)
  return remodify.strip()

df1.iloc[:,1]=df1.iloc[:,1].apply(modify_text)

print(df1.head)
print(df1.tail)

<bound method NDFrame.head of                                                 column1  \
0                                         div id access   
1                                   article name webapp   
2                   article name webapp class container   
3     header id access class container name webapp h...   
4             div id access name webapp class container   
...                                                 ...   
1738                            article class container   
1739       header id access name webapp class container   
1740  header id access class container name webapp d...   
1741                                    div name webapp   
1742                                   footer id access   

                                                column2  
0                         / / div [ @ id = ' access ' ]  
1                   / / article [ @ name = ' webapp ' ]  
2     / / article [ @ name = ' webapp ' and @ class ...  
3     / / header [ @ id = ' a

In [ ]:
new_df1 = pd.DataFrame({'column1': df1.iloc[:,0], 'Modified_column2':df1.iloc[:,1]})

print(new_df1.head())

                                             column1  \
0                                      div id access   
1                                article name webapp   
2                article name webapp class container   
3  header id access class container name webapp h...   
4          div id access name webapp class container   

                                    Modified_column2  
0                      / / div [ @ id = ' access ' ]  
1                / / article [ @ name = ' webapp ' ]  
2  / / article [ @ name = ' webapp ' and @ class ...  
3  / / header [ @ id = ' access ' and @ class = '...  
4  / / div [ @ id = ' access ' and @ name = ' web...  


In [ ]:
# Define default values for special characters and gap for words
special_characters = {'/': 5, '@': 10, '[':15, ']':20, '\'':25, '=':30,'div': 35,
    'p': 40,
    'h1': 45,
    'h2': 50,
    'h3': 55,
    'h4': 60,
    'h5': 65,
    'h6': 70,
    'a': 75,
    'img': 80,
    'span': 85,
    'ul': 90,
    'li': 95,
    'form': 100,
    'input': 105,
    'button': 110,
    'label': 115,
    'section': 120,
    'header': 125,
    'nav': 130,
    'footer': 135,
    'table': 140,
    'tr': 145,
    'td': 150,
    'th': 155,
    'article': 160,
    'aside': 165,
    'main': 170,
    'blockquote': 175,
    'cite': 180,
    'textarea': 185,
    'strong': 190,
    'iframe': 195,
    'video': 200,
    'audio': 205,
    'source': 210,
    'ol': 215,
    '()': 220,
    'b': 225,
    'canvas': 230,
    'caption': 235,
    'code': 240,
    'datalist': 245,
    'del': 250,
    'details': 255,
    'dialog': 260,
    'fieldset': 265,
    'figcaption': 270,
    'figure': 275,
    'footer': 280,
    'header': 285,
    'hr': 290,
    'select': 295,
    'ins': 300,
    'kbd': 305,
    'mark': 310,
    'meter': 315,
    'output': 320,
    'progress': 325,
    'q': 330,
    'rp': 335,
    'rt': 340,
    'ruby': 345,
    's': 350,
    'samp': 355,
    'small': 360,
    'sub': 365,
    'summary': 370,
    'sup': 375,
    'template': 380,
    'time': 385,
    'role': 390,
    'var': 395,
    'wbr': 400,
    'class': 405,
    'id': 410,
    'name': 415,
    'style': 420,
    'href': 425,
    'target': 430,
    'src': 435,
    'alt': 440,
    'type': 445,
    'value': 450,
    'for': 455,
    'action': 460,
    'method': 465,
    'title': 470,
    'placeholder': 475,
    'required': 480,
    'checked': 485,
    'disabled': 490,
    'readonly': 495,
    'size': 500,
    'maxlength': 505,
    'aria-label': 510,
    'max': 515,
    'step': 520,
    'rows': 525,
    'cols': 530,
    'autocomplete': 535,
    'autofocus': 540,
    'multiple': 545,
    'pattern': 550,
    'selected': 555,
    'labels': 560,
    'contenteditable': 565,
    'spellcheck': 570,
    'tabindex': 575,
    'download': 580,
    'rel': 585,
    'size': 590,
    'shape': 595,
    'coords': 600,
    'autoplay': 605,
    'controls': 610,
    'loop': 615,
    'muted': 620,
    'preload': 625,
    'poster': 630,
    'kind': 635,
    'srclang': 640,
    'sandbox': 645,
    'allow': 650,
    'height': 655,
    'width': 660,
    'srcdoc': 665,
    'allowfullscreen': 670,
    'allowpaymentrequest': 675,
    'async': 680,
    'charset': 685,
    'defer': 690,
    'language': 695,
    'scoped': 700,
    'wrap': 705,
    'con': 710,
    'http-equiv': 715,
    'scheme': 720,
    'usemap': 725,
    'headers': 730,
    'nowrap': 735,
    'abbr': 740,
    'scope': 745,
    'rowspan': 750,
    'colspan': 755,
    'align': 760,
    'valign': 765,
    'bgcolor': 770,
    'char': 775,
    'charoff': 780,
    'clear': 785,
    'dir': 790,
    'lang': 795,
    'xml:lang': 800,
    'compact': 805,
    'cite': 810,
    'datetime': 815,
    'start': 820,
    'reversed': 825,
    'classid': 830,
    'codebase': 835,
    'data': 840,
    'archive': 845,
    'declare': 850,
    'standby': 855,
    'vspace': 860,
    'hspace': 865,
    'axis': 870,
    'longdesc': 875,
    'border': 880,
    'frame': 885,
    'rules': 890,
    'summary': 895,
    'accept': 900,
    'accept-charset': 905,
    'enctype': 910,
    'formaction': 915,
    'formenctype': 920,
    'formmethod': 925,
    'formnovalidate': 930,
    'formtarget': 935,
    'autosave': 940,
    'contextmenu': 945,
    'draggable': 950,
    'dropzone': 955,
    'hidden': 960,
    'translate': 965,
    'alink': 970,
    'background': 975,
    'link': 980,
    'text': 985,
    '>':985,
    'vlink': 990,
    'default': 995,
    'and':1000,
    'data-test':1005,
    'time':1010
}
word_indices1 = special_characters.copy()
print(type(word_indices1))


<class 'dict'>


In [ ]:
output1 = []
num_samples = len(new_df1)


for i in range(num_samples):
  next_index_column1=2000
  column1_indices = []
  modifyColumn_indices=[]
  firstcol = new_df1.iloc[i,0]
  secondcol = new_df1.iloc[i,1]


  for word in firstcol.split():
    if word in word_indices1:
      column1_indices.append(word_indices1[word])
    else:
      word_indices1[word] = next_index_column1
      column1_indices.append(next_index_column1)
      next_index_column1 += 5

  next_index_column1=2000


  for word in secondcol.split():
    if word in word_indices1:
      modifyColumn_indices.append(word_indices1[word])

  word_indices1 = special_characters.copy()


  output1.append([column1_indices, modifyColumn_indices])

# Convert to DataFrame
df_output1 = pd.DataFrame(output1, columns=['column1', 'column2'])

# Print the DataFrame
print(df_output1)






                                                column1  \
0                                       [35, 410, 2000]   
1                                      [160, 415, 2000]   
2                           [160, 415, 2000, 405, 2005]   
3     [285, 410, 2000, 405, 2005, 415, 2010, 425, 2015]   
4                 [35, 410, 2000, 415, 2005, 405, 2010]   
...                                                 ...   
1738                                   [160, 405, 2000]   
1739             [285, 410, 2000, 415, 2005, 405, 2010]   
1740  [285, 410, 2000, 405, 2005, 415, 2010, 1005, 2...   
1741                                    [35, 415, 2000]   
1742                                   [280, 410, 2000]   

                                                column2  
0         [5, 5, 35, 15, 10, 410, 30, 25, 2000, 25, 20]  
1        [5, 5, 160, 15, 10, 415, 30, 25, 2000, 25, 20]  
2     [5, 5, 160, 15, 10, 415, 30, 25, 2000, 25, 100...  
3     [5, 5, 285, 15, 10, 410, 30, 25, 2000, 25, 100...  
4

In [ ]:
import numpy as np
max_length = 50
# Transform all samples' lengths into max length by adding zeros
df_output1.iloc[:,0] = df_output1.iloc[:,0].apply(lambda x: np.pad(x, (0, max_length - len(x)), 'constant'))
df_output1.iloc[:,1] = df_output1.iloc[:,1].apply(lambda x: np.pad(x, (0, max_length - len(x)), 'constant'))

# Display the first few rows of the updated DataFrame
# print(df_output1.head())
# print(df_output1.tail())

# print(df_output1.iloc[11987,0])
print(df_output1.iloc[59,1])

# print(df_output1.iloc[11204,0])
# print(df_output1.iloc[11204,1])

# df_output1.to_csv('output.csv', index=False)

[   5    5  280   15   10  405   30   25 2000   25   20    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [ ]:
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
input_array = np.array(df_output1.iloc[:,0].tolist())
output_array = np.array(df_output1.iloc[:,1].tolist())

print(input_array)

[[  35  410 2000 ...    0    0    0]
 [ 160  415 2000 ...    0    0    0]
 [ 160  415 2000 ...    0    0    0]
 ...
 [ 285  410 2000 ...    0    0    0]
 [  35  415 2000 ...    0    0    0]
 [ 280  410 2000 ...    0    0    0]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(input_array, output_array, test_size=0.2, random_state=42)
model = RandomForestRegressor()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(mse)

0.0


In [ ]:
# new_element_id_array = [35, 415, 2000, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0, 0, 0, 0, 0, 0, 0,0,0,0,0, 0, 0, 0, 0, 0, 0,0,0,0,0, 0, 0, 0, 0, 0, 0,0,0,0]

# new_element_id_array = np.array([new_element_id_array])

#class=405 id=410 name=415
word_indices2 = special_characters.copy()
column1_indi = []
next_index_colum = 2000
#<button class="nav-menu-desktopHamburgerSticky nav-menu-desktopHamburger nav-menu-button" aria-label="Open Navigation Menu"></button>
element1 = ['div','class','nav-menu-desktopHamburgerSticky nav-menu-desktopHamburger nav-menu-button','aria-label','Open Navigation Menu']

for item in element1:
   if item in word_indices2:
      column1_indi.append(word_indices2[item])

   else:
      word_indices2[item] = next_index_colum
      column1_indi.append(next_index_colum)
      print(next_index_colum)
      next_index_colum += 5


print(column1_indi)
count = len(column1_indi)

for i in range(50-count):
  column1_indi.append(0)

print(column1_indi)
print(len(column1_indi))

new_element_id_array = column1_indi
new_element_id_array = np.array([new_element_id_array])

#5, 5, 80, 15, 10, 410, 30, 25, 2000, 25, 20

predicted_xpath_array = model.predict(new_element_id_array.reshape(1,-1))

print("predicted Xpath array:", predicted_xpath_array)
print(len(predicted_xpath_array))
print(type(predicted_xpath_array))


rounded_numbers2 = (np.round(predicted_xpath_array / 5)* 5).astype(int)

rounded_numbers_list2= rounded_numbers2.tolist()

print(rounded_numbers_list2)
reversing2 = {value: key for key, value in word_indices2.items()}
words2 = [reversing2[number] for number in rounded_numbers2.flatten() if number != 0]

replace_this = 'and'
replace_with = ' and '

newWords2 = [replace_with if x == replace_this else x for x in words2]

print(newWords2)

print(type(newWords2))

result_xpath = ''.join(newWords2)
print(result_xpath)

2000
2005
[35, 405, 2000, 510, 2005]
[35, 405, 2000, 510, 2005, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
50
predicted Xpath array: [[   5.    5.   35.   15.   10.  405.   30.   25. 2000.   25. 1000.   10.
   415.   30.   25. 2005.   25.   20.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.]]
1
<class 'numpy.ndarray'>
[[5, 5, 35, 15, 10, 405, 30, 25, 2000, 25, 1000, 10, 415, 30, 25, 2005, 25, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
['/', '/', 'div', '[', '@', 'class', '=', "'", 'nav-menu-desktopHamburgerSticky nav-menu-desktopHamburger nav-menu-button', "'", ' and ', '@', 'name', '=', "'", 'Open Navigation Menu', "'", ']']
<class 'list'>
//div[@class='nav-menu-desktopHamburgerSticky nav-

In [ ]:

import joblib

# Assuming X_train and y_train are your training data



# Save the trained model to a file
joblib.dump(model, 'XpathCreator.pkl')

# Download the saved model file
from google.colab import files
files.download('XpathCreator.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>